<a href="https://colab.research.google.com/github/Kudryai/DataAnalyst/blob/main/2_KudryvtcevAI_VK_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Задание 1. Собираем данные с помощью Requests**

С помощью методов API ВКонтакте получите список высших учебных заведений и названия их факультетов в городе Томск.
Результат должен быть записан в файл JSON в следующем формате:

* Решение задания

In [ ]:
import requests
import json


ACCESS_TOKEN = 'Your TOKEN'


url = f"https://api.vk.com/method/database.getCities/" # Находим город Томск
params = {'access_token':ACCESS_TOKEN,
          'country_id':1,
          'q':'Томск',
          'sort':6,
          'count':1,
          'v':5.131}
res = requests.get(url, params=params)
res_city = res.json()

url = f"https://api.vk.com/method/database.getUniversities/" # Находим все университеты Томска
params1 = {'access_token':ACCESS_TOKEN,
          'country_id':1,
          'city_id':144,
          'v':5.131}
res = requests.get(url, params=params1)
res_univer = res.json()

if res_city.get('response',''):
    city = res_city['response']['items']
    univer = res_univer['response']['items']
    result = [{
            'id': bam['id'],
            'name': bam['title'],
            'universites': [{
                'id':uni['id'],
                'name':uni['title'],
                'faculties':[{
                    'name': facul['title']
                }for facul in requests.get('https://api.vk.com/method/database.getFaculties/', 
                                                       params={'access_token':ACCESS_TOKEN,
                                                               'university_id':uni['id'],
                                                               'v':5.131}).json()['response']['items']]
                } for uni in univer]
        } for bam in city]
    with open('UniverInfo.json','w') as outfile:
        json.dump(result, outfile, ensure_ascii=False, indent=4)
else:
    print(f'Errors mining data')

# **Задание 2. Библиотека VK API**

Напишите код, который получает список названия школ города Кемерово с помощью библиотеки vk_api и записывает результаты в файл JSON.

In [ ]:
!pip install vk_api # Установка vk_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 4.0 MB/s 


In [ ]:
import vk_api
import csv

vk_session = vk_api.VkApi(token=ACCESS_TOKEN)
vk = vk_session.get_api()

city = vk.database.getCities(country_id=1,q='Кемерово', count=1)
city_id = city['items'][0]['id'] if city ['count'] > 0 else None

if city_id:
        schools = vk.database.getSchools(country_id=1, city_id=city_id)
        schools_name = [{'city': cit['title'],
                                     'schools':[{'name': sch['title']} for sch in schools['items']]} for cit in city['items']]
        with open('SchoolsInfo.json','w') as outfile:
          json.dump(schools_name, outfile, ensure_ascii=False, indent=4)



# **Задание 3. Запись данных в CSV формат**

С помощью методов API «ВКонтакте» получите 1000 подписчиков группы «Лентач», отсортирванных по дате регистрации.

Вам необходимо собрать следующие данные в CSV файл: пол, название город, семейное положение (ФИО партнера не указывать).

In [ ]:
import vk_api
import csv


vk_session = vk_api.VkApi(token=ACCESS_TOKEN)
vk = vk_session.get_api()

lnt4 = vk.groups.search(q='Лентач', count=1)
lnt4_id = lnt4['items'][0]['id']
lnt4_info = vk.groups.getMembers(group_id=lnt4_id, count=1000, fields=['sex', 'city', 'relation'], sort='id_asc')
with open('Lenta4_info_users.csv','w',encoding='utf-8') as outfile:
            writer = csv.DictWriter(outfile, fieldnames=['sex', 'city' ,'relation'],extrasaction = 'ignore')
            writer.writeheader()
            for lnt in lnt4_info['items']:
              writer.writerow(lnt)
